In [17]:
import re
import numpy as np
from functools import cached_property

In [2]:
input_path_test = './day4_input_test.txt'
# input_path_test_b = './day2_input_test_b.txt'
input_path = './day4_input.txt'

In [29]:
class Card:
    def __init__(self, card_str):
        self.card_str = card_str
        
    @property
    def card_id(self):
        return int(self.card_str.split(":")[0].split(" ")[1])
    
    @cached_property
    def winning_numbers(self):
        winning_numbers = self.card_str.split(" | ")[0].split(": ")[1].split()
        winning_numbers = [int(w) for w in winning_numbers]
        winning_numbers.sort()
        return winning_numbers
    
    @cached_property
    def my_numbers(self):
        my_numbers = self.card_str.split(" | ")[1].split()
        my_numbers = [int(w) for w in my_numbers]
        my_numbers.sort()
        return my_numbers
    
    @property
    def find_matches(self):
        winning_idx = 0
        my_idx = 0
        matches = []
        while winning_idx < len(self.winning_numbers) and my_idx < len(self.my_numbers):
            winning_num = self.winning_numbers[winning_idx]
            my_num = self.my_numbers[my_idx]
            if winning_num < my_num:
                winning_idx += 1
            elif my_num < winning_num:
                my_idx += 1
            else:
                matches.append(winning_num)
                winning_idx += 1
                my_idx += 1
        return matches
    
    @property
    def value(self):
        num_matches = len(self.find_matches)
        if num_matches == 0:
            return 0
        else:
            return 2 ** (num_matches - 1)
        
class Pile:
    def __init__(self, file_path):
        self.file_path = file_path
        
    @property
    def value(self):
        cumsum = 0
        with open(self.file_path, 'r') as f:
            for line in f:
                line = line.rstrip('\n')
                card = Card(line)
                cumsum += card.value
        return cumsum

In [30]:
card = Card("Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53")
card.card_id
card.winning_numbers
card.my_numbers
card.find_matches
card.value

8

In [32]:
Pile('./day4_input_test.txt').value

13

In [33]:
Pile('./day4_input.txt').value

25174

In [102]:
class CardB:
    def __init__(self, card_str):
        self.card_str = card_str
        
    @property
    def card_id(self):
        return int(self.card_str.split(":")[0].split()[1])
    
    @cached_property
    def winning_numbers(self):
        winning_numbers = self.card_str.split(" | ")[0].split(": ")[1].split()
        winning_numbers = [int(w) for w in winning_numbers]
        winning_numbers.sort()
        return winning_numbers
    
    @cached_property
    def my_numbers(self):
        my_numbers = self.card_str.split(" | ")[1].split()
        my_numbers = [int(w) for w in my_numbers]
        my_numbers.sort()
        return my_numbers
    
    @property
    def find_matches(self):
        winning_idx = 0
        my_idx = 0
        matches = []
        while winning_idx < len(self.winning_numbers) and my_idx < len(self.my_numbers):
            winning_num = self.winning_numbers[winning_idx]
            my_num = self.my_numbers[my_idx]
            if winning_num < my_num:
                winning_idx += 1
            elif my_num < winning_num:
                my_idx += 1
            else:
                matches.append(winning_num)
                winning_idx += 1
                my_idx += 1
        return matches
    
    @property
    def tickets_won(self):
        num_matches = len(self.find_matches)
        return [self.card_id + 1 + ii for ii in range(num_matches)]
        
class PileB:
    def __init__(self, file_path):
        self.file_path = file_path
        
    @property
    def winnings(self):
        winnings_dict = {}
        with open(self.file_path, 'r') as f:
            for line in f:
                line = line.rstrip('\n')
                # print(line)
                card = CardB(line)
                winnings_dict[card.card_id] = card.tickets_won
        return winnings_dict
    
    def propogate_winnings(self, ticket_id):
        if ticket_id in self.propogated_winnings.keys():
            return self.propogated_winnings[ticket_id]
        total_dict = {}
        for ticket in self.winnings[ticket_id]:
            total_dict[ticket] = total_dict.get(ticket,0) + 1
            extra_dict = self.propogate_winnings(ticket)
            for key, value in extra_dict.items():
                total_dict[key] = total_dict.get(key, 0) + value
        self.propogated_winnings[ticket_id] = total_dict
        return total_dict
    
    @cached_property
    def propogated_winnings(self):
        return {}
    
    def value(self):
        cumsum = 0
        for ticket in self.winnings.keys():
            prop = self.propogate_winnings(ticket)
            cumsum += sum(prop.values()) + 1
        return cumsum
        
            
        

In [103]:
pile_b = PileB('./day4_input_test.txt')

In [104]:
pile_b.winnings

{1: [2, 3, 4, 5], 2: [3, 4], 3: [4, 5], 4: [5], 5: [], 6: []}

In [105]:
pile_b.propogate_winnings(1)

{2: 1, 3: 2, 4: 4, 5: 7}

In [106]:
pile_b.value()

30

In [107]:
PileB('./day4_input.txt').value()

6420979